CREAR UN AGENTE DE IA CON LANGCHAIN

Instalación de librerías:

pip install --upgrade --quiet sqlalchemy langchain langchain-community langchain-openai pandas python-dotenv
                   

In [1]:
import sqlalchemy 
import langchain 
import langchain_community
import langchain_openai
import pandas as pd
import dotenv

CARGAR DATOS CSV

In [7]:
# cargar csv en pandas
df = pd.read_csv("Amazon Sale Report.csv") 
print(df.head(7))

   index             Order ID      Date                        Status  \
0      0  405-8078784-5731545  04-30-22                     Cancelled   
1      1  171-9198151-1101146  04-30-22  Shipped - Delivered to Buyer   
2      2  404-0687676-7273146  04-30-22                       Shipped   
3      3  403-9615377-8133951  04-30-22                     Cancelled   
4      4  407-1069790-7240320  04-30-22                       Shipped   
5      5  404-1490984-4578765  04-30-22                       Shipped   
6      6  408-5748499-6859555  04-30-22                       Shipped   

  Fulfilment Sales Channel  ship-service-level    Style              SKU  \
0   Merchant      Amazon.in           Standard   SET389   SET389-KR-NP-S   
1   Merchant      Amazon.in           Standard  JNE3781  JNE3781-KR-XXXL   
2     Amazon      Amazon.in          Expedited  JNE3371    JNE3371-KR-XL   
3   Merchant      Amazon.in           Standard    J0341       J0341-DR-L   
4     Amazon      Amazon.in        

C:\Users\ander\AppData\Local\Temp\ipykernel_15720\2379400652.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Amazon Sale Report.csv")
